In [184]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [185]:
data = pd.read_csv('movie_bd_v5.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 140.3+ KB


# Предобработка

In [4]:
answers = {} # Создадим словарь для ответов

# The time given in the dataset is in string format.
# So we need to change this in datetime format
data.release_date = pd.to_datetime(data['release_date'])

# Добавим информацию о прибыльности фильма (колонка profit)
def profit_calc(rev, budg):
    return rev-budg
profit = pd.Series(list(map(profit_calc, data.revenue, data.budget)),
                   name = 'profit')
data = pd.concat([data.loc[:, :'revenue'], profit, 
                  data.loc[:, 'original_title':]], axis = 1)
data.sample(5)

,imdb_id,budget,revenue,profit,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1576,tt0289879,13000000,96060858,83060858,The Butterfly Effect,Ashton Kutcher|Amy Smart|William Lee Scott|Eld...,Eric Bress|J. Mackye Gruber,"Such minor changes, such huge consequences.",A young man struggles to access sublimated chi...,113,Science Fiction|Thriller,New Line Cinema|FilmEngine|Katalyst Films|Bend...,2004-01-22,7.1,2004
771,tt1625346,12000000,22939027,10939027,Young Adult,Charlize Theron|Patton Oswalt|Patrick Wilson|E...,Jason Reitman,Everyone gets old. Not everyone grows up.,A divorced writer from the Midwest returns to ...,94,Comedy|Drama|Romance,Paramount Pictures|Denver and Delilah Producti...,2011-12-16,5.2,2011
1392,tt0358273,28000000,186438883,158438883,Walk the Line,Joaquin Phoenix|Reese Witherspoon|Ginnifer Goo...,James Mangold,Love is a burning thing.,A chronicle of country music legend Johnny Cas...,136,Drama|Music|Romance,Tree Line Films|Konrad Pictures|Catfish Produc...,2005-09-13,7.0,2005
1661,tt0315824,50000000,6808550,-43191450,The Big Bounce,Owen Wilson|Morgan Freeman|Gary Sinise|Sara Fo...,George Armitage,It's all in who you trust,A small-time con artist and a Hawaiian real es...,88,Comedy|Crime,Warner Bros. Pictures|Shangri-La Entertainment...,2004-01-30,4.9,2004
765,tt1563738,15000000,59389433,44389433,One Day,Anne Hathaway|Jim Sturgess|Patricia Clarkson|R...,Lone Scherfig,Twenty years. Two people.,"A romantic comedy centered on Dexter and Emma,...",107,Drama|Romance,Film 4|Color Force,2011-03-02,6.9,2011


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [5]:
# В словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# Запишите свой вариант ответа
# Если ответили верно, можете добавить комментарий со значком "+"

answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

In [6]:
# Тут пишем ваш код для решения данного вопроса:
data[data.budget == data.budget.max()]

,imdb_id,budget,revenue,profit,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
723,tt1298650,380000000,1021683000,641683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,2011-05-11,6.3,2011


ВАРИАНТ 2

In [7]:
# Можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [8]:
# Длительность фильма указана в колонке runtime
title_info = data[data.runtime == data.runtime.max()].original_title
imdb_id_info = data[data.runtime == data.runtime.max()].imdb_id

# Приводим данные к строке
title_2 = title_info.to_string(index = False)[1:]
imdb_id_2 = imdb_id_info.to_string(index = False)[1:]

# Ответ передаем в переменную answer_2
answer_2 = '{title} ({imdb_id})'.format(title=title_2, imdb_id=imdb_id_2)

In [9]:
# добавляем ответ в словарь
answers['2'] = answer_2
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)'}

# 3. Какой из фильмов самый короткий (в минутах)?





In [10]:
# Длительность фильма указана в колонке runtime
title_info = data[data.runtime == data.runtime.min()].original_title
imdb_id_info = data[data.runtime == data.runtime.min()].imdb_id

# Приводим данные к строке
title_3 = title_info.to_string(index = False)[1:]
imdb_id_3 = imdb_id_info.to_string(index = False)[1:]

# Ответ передаем в переменную answer_3
answer_3 = '{title} ({imdb_id})'.format(title=title_3, imdb_id=imdb_id_3)

In [11]:
answers['3'] = answer_3
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)'}

# 4. Какова средняя длительность фильмов?


In [12]:
# Не мудрувствуя лукаво
answers['4'] = str(round(data.runtime.mean()))
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110}

# 5. Каково медианное значение длительности фильмов? 

In [13]:
answers['5'] = str(round(data.runtime.median()))

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [14]:
# Бюджет фильма указан в колонке profit
title_info = data[data.profit == data.profit.max()].original_title
imdb_id_info = data[data.profit == data.profit.max()].imdb_id

# Приводим данные к строке
title_6 = title_info.to_string(index = False)[1:]
imdb_id_6 = imdb_id_info.to_string(index = False)[1:]

# Ответ передаем в переменную answer_6
answer_6 = '{title} ({imdb_id})'.format(title=title_6, imdb_id=imdb_id_6)
answers['6'] = answer_6
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)'}

# 7. Какой фильм самый убыточный? 

In [15]:
# Бюджет фильма указан в колонке profit
title_info = data[data.profit == data.profit.min()].original_title
imdb_id_info = data[data.profit == data.profit.min()].imdb_id

# Приводим данные к строке
title_7 = title_info.to_string(index = False)[1:]
imdb_id_7 = imdb_id_info.to_string(index = False)[1:]

# Ответ передаем в переменную answer_6
answer_7 = '{title} ({imdb_id})'.format(title=title_7, imdb_id=imdb_id_7)
answers['7'] = answer_7
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)'}

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [16]:
answer_8 = data[data.revenue > data.budget].imdb_id.nunique()
answers['8'] = str(answer_8)

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [17]:
# Опрделяем фильмы 2008 года
movies = data[data.release_year == 2008]

# "Кассовость" фильма указана в колонке revenue
title_info = movies[movies.revenue == movies.revenue.max()].original_title
imdb_id_info = movies[movies.revenue == movies.revenue.max()].imdb_id

title_9 = title_info.to_string(index = False)[1:]
imdb_id_9 = imdb_id_info.to_string(index = False)[1:]

# Ответ передаем в переменную answer_9
answer_9 = '{title} ({imdb_id})'.format(title=title_9, imdb_id=imdb_id_9)
answers['9'] = answer_9
answers


{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)'}

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [18]:
# Опрделяем фильмы 20012 - 2014 годов
movies = data[(data.release_year >= 2012) & (data.release_year <= 2014)]

title_info = movies[movies.profit == movies.profit.min()].original_title
imdb_id_info = movies[movies.profit == movies.profit.min()].imdb_id

title_10 = title_info.to_string(index = False)[1:]
imdb_id_10 = imdb_id_info.to_string(index = False)[1:]

# Ответ передаем в переменную answer_9
answer_10 = '{title} ({imdb_id})'.format(title=title_10, imdb_id=imdb_id_10)
answers['10'] = answer_10
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)'}

# 11. Какого жанра фильмов больше всего?

In [19]:
genres = pd.Series(data.genres.str.cat(sep='|').split('|'))
answer_11 = genres.value_counts().index[0]
answers['11'] = answer_11
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama'}

ВАРИАНТ 2

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [20]:
counter = Counter() # счетчик 'прибыльных' жанров
profit_genres = data[data.profit > 0].genres.str.split('|')
for line in profit_genres:
    for genre in line:
        counter[genre]+=1 
counter.most_common(1)

answers['12'] = 'Drama'


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [21]:
directors_rev = data.groupby(['director'])[['revenue']].sum()

answers['13'] = directors_rev.sort_values(['revenue'],
                                      ascending = False).iloc[0].name


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [22]:
# Фильмы в жанре Action
action_mov = data[data.genres.str.contains('Action')]
# И их режиссеры
action_dir = action_mov.director.str.split('|').explode()

answers['14'] = action_dir.value_counts().index[0]
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez'}

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [182]:
# Развернем датасет относительно cast с помощью ф-ции explode
cast_set = data[data.release_year == 2012].copy()
cast_set.cast = cast_set.cast.str.split('|')
cast_expl = cast_set.explode('cast')

# Сгруппируем по актерам/актрисам
cast_rev = cast_expl.groupby('cast')[['revenue']].sum()

answers['15'] = cast_rev.sort_values('revenue', ascending = False).index[0]
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'September',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '27': "'Daniel Radcliffe', 'Emma Watson'"}

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [24]:
# Развернем датасет относительно cast с помощью ф-ции explode
cast_set = data.copy()
cast_set.cast = cast_set.cast.str.split('|')
cast_expl = cast_set.explode('cast')
high_budg = cast_expl[cast_expl.budget > data.budget.mean()]

answers['16'] = high_budg.cast.value_counts().index[0]
answers
    

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Emma Watson',
 '16': 'Matt Damon'}

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [25]:
# Найдем фильмы Николаса и 'развернем' информацию об их жанрах
nicolas = data[data.cast.str.contains('Nicolas Cage')].copy()
nicolas.genres = nicolas.genres.str.split('|')
nicolas = nicolas.explode('genres')

answers['17'] = nicolas.genres.value_counts().index[0]
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Emma Watson',
 '16': 'Matt Damon',
 '17': 'Action'}

# 18. Самый убыточный фильм от Paramount Pictures

In [26]:
pp = data[data.production_companies.str.contains('Paramount Pictures')]

title_18 = pp.sort_values('profit').original_title.iloc[0]
imdb_id_18 = pp.sort_values('profit').imdb_id.iloc[0]
answers['18'] = '{title} ({imdb_id})'.format(title=title_18, imdb_id=imdb_id_18)
answers


{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Emma Watson',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)'}

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [27]:
years_rev = data.groupby('release_year')['revenue'].sum()

answers['19'] = str(years_rev.sort_values().index[-1])
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Emma Watson',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015'}

# 20. Какой самый прибыльный год для студии Warner Bros?

In [28]:
wb = data[data.production_companies.str.contains('Warner Bros')]

wb_profit = wb.groupby('release_year')['profit'].sum()
answers['20'] = str(wb_profit.sort_values().index[-1])
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Emma Watson',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014'}

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [29]:
mov_by_month = data.release_date.dt.month_name()
answers['21'] = str(mov_by_month.value_counts().index[0])
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Emma Watson',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'September'}

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [30]:
answers['22'] = str(len(data.query('release_date.dt.month in [6, 7, 8]')))
answers


{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Emma Watson',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'September',
 '22': '450'}

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [31]:
# Найдем 'зимние' фильмы и их режиссеров
winter_directors = data.query('release_date.dt.month in [1, 2, 12]').director

answers['23'] = winter_directors.str.split('|').explode().value_counts().index[0]
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Emma Watson',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'September',
 '22': '450',
 '23': 'Peter Jackson'}

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [32]:
# Раскрываем информацию о production companies
data_prod = data[['original_title', 'production_companies']].copy()
data_prod.production_companies = data_prod.production_companies.str.split('|')
data_prod=data_prod.explode('production_companies')

# Вместо названий фильмов укажем их длину...
data_prod.original_title = data_prod.original_title.apply(lambda x: len(x))

# ...и найдем искомую студию
prod_titles = data_prod.groupby('production_companies').mean()

answers['24'] = prod_titles.sort_values('original_title').index[-1]
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Emma Watson',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'September',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions'}

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [181]:
# Раскрываем информацию о production companies
data_prod = data[['overview', 'production_companies']].copy()
data_prod.production_companies = data_prod.production_companies.str.split('|')
data_prod=data_prod.explode('production_companies')

# Вместо аннотаций укажем их длину...
data_prod.overview = data_prod.overview.apply(lambda x: len(x))

# ...и найдем искомую студию
prod_overview = data_prod.groupby('production_companies').mean()

answers['25'] = prod_overview.sort_values('overview').index[-1]
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Emma Watson',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'September',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '27': "'Daniel Radcliffe', 'Emma Watson'"}

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [34]:
ratings = data[['original_title', 'vote_average']]
# 1% лучших фильмов имеют следующий рейтинг:
percentile_99 = float(ratings.describe(percentiles = [0.99]).loc['99%'])
ratings[ratings.vote_average > percentile_99].sort_values('vote_average',
                                                     ascending = False)


,original_title,vote_average
599,The Dark Knight,8.1
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
125,The Imitation Game,8.0
119,Guardians of the Galaxy,7.9
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
872,The Pianist,7.9


In [186]:
answers['26'] = 'The Dark Knight, Inside Out, 12 Years a Slave'
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'September',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '27': "'Daniel Radcliffe', 'Emma Watson'",
 '26': 'The Dark Knight, Inside Out, 12 Years a Slave'}

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [172]:

def to_list(x):
'''Преобразовывем Series в список'''
    y=x.tolist()
    return y

# Развернем датасет относительно cast с помощью ф-ции explode
cast_set = data.copy()
cast_set.cast = cast_set.cast.str.split('|')
cast_set = cast_set.explode('cast')[['cast', 'imdb_id']]
cast_set = cast_set.groupby('cast').agg(to_list)
cast_set
fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
# Дальше идет составление сводной таблицы с каким-то aggfunc (не додумался)

# А еще дальше по это таблице я определяю, правильный ответ

,imdb_id
cast,
50 Cent,"[tt1798684, tt1034331, tt1925431, tt1211956, t..."
A.J. Cook,[tt0309593]
Aaliyah,"[tt0238546, tt0165929]"
Aamir Khan,[tt1187043]
Aaran Thomas,[tt0367959]
...,...
ZoÃ« Bell,[tt0462322]
ZoÃ« Borde,[tt2975578]
ZoÃ« Kravitz,[tt1815862]


Вариант 2

In [187]:
combs = [] 
for cast in data.cast.str.split('|'):
    combs.extend(combinations(cast, 2))
answers['27'] = str(pd.Series(combs).value_counts().index[0])[1:-1]
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'September',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '27': "'Daniel Radcliffe', 'Emma Watson'",
 '26': 'The Dark Knight, Inside Out, 12 Years a Slave'}

# Submission

In [188]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'September',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '27': "'Daniel Radcliffe', 'Emma Watson'",
 '26': 'The Dark Knight, Inside Out, 12 Years a Slave'}

In [189]:
# и убедиться что ни чего не пропустил)
len(answers)

27